###  Implement of Optimus pipeline
#### Pipeline version v6.0.0 with output in h5ad format

#### 1. The wld files used in this pipeline are downloaded from the following link:

###### https://github.com/broadinstitute/warp/blob/develop/pipelines/skylab/optimus/Optimus.wdl
###### https://github.com/broadinstitute/warp/blob/develop/tasks/skylab/H5adUtils.wdl
###### https://github.com/broadinstitute/warp/blob/develop/tasks/skylab/FastqProcessing.wdl
###### https://github.com/broadinstitute/warp/blob/develop/tasks/skylab/StarAlign.wdl
###### https://github.com/broadinstitute/warp/blob/develop/tasks/skylab/Metrics.wdl
###### https://github.com/broadinstitute/warp/blob/develop/tasks/skylab/RunEmptyDrops.wdl
###### https://github.com/broadinstitute/warp/blob/develop/tasks/skylab/CheckInputs.wdl
###### https://github.com/broadinstitute/warp/blob/develop/tasks/skylab/MergeSortBam.wdl
###### https://github.com/broadinstitute/CellBender/blob/master/wdl/cellbender_remove_background.wdl

#### recent update information
###### Optimus.wdl: Aug 31, 2023 pipeline_version = "6.0.0"
###### H5adUtils.wdl: Aug 28, 2023
###### StarAlign.wdl: Aug 28, 2023
###### Metrics.wdl: Aug 31, 2023
###### CellBender v0.3.0: Aug 8, 2023

A comprehensive gene annotation (PRI) is used in this pipeline: https://www.gencodegenes.org/human/release_27.html

#### 2. Check the difference between the wdl files in pipeline_version 5.7.4 and 6.0.0

###### The pipeline version 5.7.4 is in branch sc-pip and the 6.0.0 is in branch sc-pip-v2
###### The following wld files are named with version information for the comparison, the pipeline, the version information (_v6.0.0 will be removed in the future)

In [12]:
### current directory ###
!pwd

/Users/xiliu/Documents/analysis/terraPipelines/notebook


In [1]:
!diff ../scAtlas/optimus/Optimus_v6.0.0.wdl  ../scAtlas/optimus/Optimus.wdl

6a7
> import "../tasks/LoomUtils.wdl" as LoomUtils
9d9
< import "../tasks/H5adUtils.wdl" as H5adUtils
11a12
> 
13c14
<     description: "The optimus 3' pipeline processes 10x genomics sequencing data based on the v2 chemistry. It corrects cell barcodes and UMIs, aligns reads, marks duplicates, and returns data as alignments in BAM format and as counts in sparse matrix exchange format."
---
>     description: "The optimus 3' pipeline processes 10x genomics sequencing data based on v2 or v3 chemistry. It corrects cell barcodes and UMIs, aligns reads, marks duplicates, and returns data as alignments in BAM format and as counts in sparse matrix exchange format."
40,42d40
<     # read_structure is based on v2 or v3 chemistry
<     String read_struct = checkOptimusInput.read_struct_out
< 
53,54c51,52
<     # Set to Forward, Reverse, or Unstranded to account for stranded library preparations (per STARsolo documentation)
<     String star_strand_mode = "Forward"
---
>     # Set to true to coun

In [3]:
!diff ../scAtlas/tasks/checkInputs_v6.0.0.wdl ../scAtlas/tasks/checkInputs.wdl 

111c111
<     # Check for chemistry version to produce read structure and whitelist
---
>     
116d115
<       echo 16C10M > read_struct.txt
121d119
<       echo 16C12M > read_struct.txt
153d150
<     String read_struct_out = read_string("read_struct.txt")


In [3]:
!diff ../scAtlas/tasks/FastqProcessing_v6.0.0.wdl ../scAtlas/tasks/FastqProcessing.wdl 

11d10
<     String read_struct
14c13
<     String docker = "us.gcr.io/broad-gotc-prod/warp-tools:1.0.1-1686932671"
---
>     String docker = "us.gcr.io/broad-gotc-prod/warp-tools:1.0.1-1679490798"
33d31
<     read_struct: "read structure for the 10x chemistry. This automatically selected in the checkInputs task"
105a104,105
>         --barcode-length 16 \
>         --umi-length $UMILENGTH \
109d108
<         --read-structure "~{read_struct}" \
140c139
<     String docker =  "us.gcr.io/broad-gotc-prod/warp-tools:1.0.1-1686932671"
---
>     String docker =  "quay.io/humancellatlas/secondary-analysis-sctools:v0.3.14-test2"
234,261d232
< 
< task FastqProcessATAC {
< 
<     input {
<         Array[String] read1_fastq
<         Array[String] read3_fastq
<         Array[String] barcodes_fastq
<         String read_structure = "16C"
<         String barcode_orientation = "FIRST_BP_RC"
<         String output_base_name
<         File whitelist
< 
<         # [?] copied from corresponding optimu

In [4]:
!diff ../scAtlas/tasks/Metrics_v6.0.0.wdl ../scAtlas/tasks/Metrics.wdl

12c12
<     String docker = "us.gcr.io/broad-gotc-prod/warp-tools:1.0.6-1692962087"
---
>     String docker = "us.gcr.io/broad-gotc-prod/warp-tools:1.0.1-1679941323"
62c62,63
<     mv updated.~{input_id}.cell-metrics.csv ~{input_id}.cell-metrics.csv
---
>     # remove the following columns: reads_unmapped, reads_mapped_exonic, reads_mapped_intronic, reads_mapped_utr, duplicate_reads, reads_mapped_intergenic
>     cut -d',' -f 1-4,8-9,11-26,29-36 updated.~{input_id}.cell-metrics.csv > ~{input_id}.cell-metrics.csv
88c89
<     String docker = "us.gcr.io/broad-gotc-prod/warp-tools:1.0.5-1692706846"
---
>     String docker = "us.gcr.io/broad-gotc-prod/warp-tools:1.0.1-1679941323"
128c129,130
<     mv updated.~{input_id}.gene-metrics.csv ~{input_id}.gene-metrics.csv
---
>     # remove the following columns: reads_mapped_exonic, reads_mapped_intronic, reads_mapped_utr, duplicate_reads
>     cut -d',' -f 1-4,8-9,11-27 updated.~{input_id}.gene-metrics.csv > ~{input_id}.gene-metrics.csv
265c267,

In [5]:
!diff ../scAtlas/tasks/StarAlign_v6.0.0.wdl ../scAtlas/tasks/StarAlign.wdl

222d221
<     String star_strand_mode
228c227
<     String docker = "us.gcr.io/broad-gotc-prod/star:1.0.1-2.7.11a-1692706072"
---
>     String docker = "us.gcr.io/broad-gotc-prod/star:1.0.0-2.7.9a-1658781884"
245d243
<     star_strand_mode: "STAR mode for handling stranded reads. Options are 'Forward', 'Reverse, or 'Unstranded'"
281c279
<         COUNTING_MODE="GeneFull_Ex50pAS"
---
>         COUNTING_MODE="GeneFull"
286,294d283
< # Check that the star strand mode matches STARsolo aligner options
<     if [[ "~{star_strand_mode}" == "Forward" ]] || [[ "~{star_strand_mode}" == "Reverse" ]] || [[ "~{star_strand_mode}" == "Unstranded" ]]
<     then
<         ## single cell or whole cell
<         echo STAR mode is assigned
<     else
<         echo Error: unknown STAR strand mode: "~{star_strand_mode}". Should be Forward, Reverse, or Unstranded.
<         exit 1;
<     fi
307c296
<       --soloStrand ~{star_strand_mode} \
---
>       --soloStrand Unstranded \
320,322c309,310
<       --out

In [7]:
!diff ../scAtlas/tasks/MergeSortBam_v6.0.0.wdl ../scAtlas/tasks/MergeSortBam.wdl

In [6]:
!diff ../scAtlas/tasks/RunEmptyDrops_v6.0.0.wdl ../scAtlas/tasks/RunEmptyDrops.wdl

diff: ../scAtlas/tasks/RunEmptyDrops_v6.0.0.wdl: No such file or directory


#### 3. Pipeline validation

In [15]:
### Syntex validation ###
!java -jar ../../womtool/womtool-85.jar validate -l ../scAtlas/optimus/Optimus.wdl

Success!
List of Workflow dependencies is:
/Users/xiliu/Documents/analysis/terraPipelines/scAtlas/tasks/CheckInputs.wdl
/Users/xiliu/Documents/analysis/terraPipelines/scAtlas/tasks/FastqProcessing.wdl
/Users/xiliu/Documents/analysis/terraPipelines/scAtlas/tasks/RunEmptyDrops.wdl
/Users/xiliu/Documents/analysis/terraPipelines/scAtlas/tasks/cellbender_remove_background.wdl
/Users/xiliu/Documents/analysis/terraPipelines/scAtlas/tasks/StarAlign.wdl
/Users/xiliu/Documents/analysis/terraPipelines/scAtlas/tasks/H5adUtils.wdl
/Users/xiliu/Documents/analysis/terraPipelines/scAtlas/tasks/MergeSortBam.wdl
/Users/xiliu/Documents/analysis/terraPipelines/scAtlas/tasks/Metrics.wdl


In [20]:
### Running the workflow
!java -jar ../../cromwell/cromwell-85.jar run ../scAtlas/optimus/Optimus.wdl -i ../scAtlas/optimus/test_data/10k_pbmc_v3_outputs.json

[2023-09-20 11:08:52,00] [info] Running with database db.url = jdbc:hsqldb:mem:a11f2969-e780-4c5f-bcd0-4ef809066ba0;shutdown=false;hsqldb.tx=mvcc
[2023-09-20 11:08:53,18] [info] Running migration RenameWorkflowOptionsInMetadata with a read batch size of 100000 and a write batch size of 100000
[2023-09-20 11:08:53,20] [info] [RenameWorkflowOptionsInMetadata] 100%
[2023-09-20 11:08:53,30] [info] Running with database db.url = jdbc:hsqldb:mem:95187750-31e2-4b5c-8805-e8786fac8cd7;shutdown=false;hsqldb.tx=mvcc
[2023-09-20 11:08:53,48] [info] Slf4jLogger started
[2023-09-20 11:08:53,56] [info] Workflow heartbeat configuration:
{
  "cromwellId" : "cromid-cfcc906",
  "heartbeatInterval" : "2 minutes",
  "ttl" : "10 minutes",
  "failureShutdownDuration" : "5 minutes",
  "writeBatchSize" : 10000,
  "writeThreshold" : 10000
}
[2023-09-20 11:08:53,58] [info] Metadata summary refreshing every 1 second.
[2023-09-20 11:08:53,58] [info] No metadata archiver defined in config
[2023-09-20 11:08:53,58] [

In [21]:
!java -jar ../../cromwell/cromwell-85.jar run ../scAtlas/optimus/Optimus.wdl -i ../scAtlas/optimus/test_data/inputs.json
#inputs.json is downloaded directly form terra.bio

[2023-09-20 11:09:04,54] [info] Running with database db.url = jdbc:hsqldb:mem:d9a59d38-cd15-4de2-a6fb-b9a832f6839b;shutdown=false;hsqldb.tx=mvcc
[2023-09-20 11:09:05,72] [info] Running migration RenameWorkflowOptionsInMetadata with a read batch size of 100000 and a write batch size of 100000
[2023-09-20 11:09:05,73] [info] [RenameWorkflowOptionsInMetadata] 100%
[2023-09-20 11:09:05,85] [info] Running with database db.url = jdbc:hsqldb:mem:cbf3ff03-2e69-4803-b218-4eb50c03133e;shutdown=false;hsqldb.tx=mvcc
[2023-09-20 11:09:06,04] [info] Slf4jLogger started
[2023-09-20 11:09:06,12] [info] Workflow heartbeat configuration:
{
  "cromwellId" : "cromid-b1aec6f",
  "heartbeatInterval" : "2 minutes",
  "ttl" : "10 minutes",
  "failureShutdownDuration" : "5 minutes",
  "writeBatchSize" : 10000,
  "writeThreshold" : 10000
}
[2023-09-20 11:09:06,14] [info] Metadata summary refreshing every 1 second.
[2023-09-20 11:09:06,15] [info] No metadata archiver defined in config
[2023-09-20 11:09:06,15] [

#### 4. Data Loading into terra

In [3]:
%load_ext autoreload
%autoreload 2

In [2]:
from basicAtlas import terra

In [1]:
# Variables definition
GCP_BUCKET="gs://whitelabgx-references"
GCP_FOLDER_NAME="resources/pbmc_10k_10X_v3"
TERRA_WS = 'whitelabgx/scRNAseq'
PROJECT="optimus_V6.0.0_wlg"

In [6]:
# Uploaded samples in google bucket associated with terra workspace
terra.uploadFromFolder_10x(GCP_BUCKET + '/' + GCP_FOLDER_NAME + '/',
                           TERRA_WS,
                           samplesetname=PROJECT,
                           fformat="fastqR1R2",
                           sep='_00|_00',
                           loc =0,
                           test=True) # test=True to test the function after upload successfully of the files to terra

please be sure you gave access to your terra email account access to this bucket


,r1_fastq,r2_fastq,i1_fastq,Source,participant
sample_id,,,,,
pbmc_10k_v3_S1_L001,gs://whitelabgx-references/resources/pbmc_10k_...,gs://whitelabgx-references/resources/pbmc_10k_...,gs://whitelabgx-references/resources/pbmc_10k_...,U,pbmc_10k_v3_S1
pbmc_10k_v3_S1_L002,gs://whitelabgx-references/resources/pbmc_10k_...,gs://whitelabgx-references/resources/pbmc_10k_...,gs://whitelabgx-references/resources/pbmc_10k_...,U,pbmc_10k_v3_S1
